# Imports

In [1]:
import pandas as pd

# Read CSV

In [2]:
listings = pd.read_csv('og_data/listings2024.csv')

# Cleaning

## 1. Cleaning bathrooms_text -> bathrooms & bathroom_type

In [3]:
def parse_bathrooms(bathrooms_text):
    if pd.isnull(bathrooms_text): 
        return pd.Series([0.0, "unknown"]) # Return 0.0 if the value is missing
    bathrooms_text = bathrooms_text.lower() # Convert to lowercase
    if "half" in bathrooms_text:
        return pd.Series([0.5, "shared" if "shared" in bathrooms_text else "private"]) # Return 0.5 if it's a half bathroom 
    elif "shared" in bathrooms_text:
        return pd.Series([float(bathrooms_text.split(" ")[0]), "shared"]) # Return the first number if it's a shared bathroom
    else:
        return pd.Series([float(bathrooms_text.split(" ")[0]), "private"]) # Return the first number if it's a private bathroom

# Apply the function
listings[['bathrooms', 'bathroom_type']] = listings['bathrooms_text'].apply(parse_bathrooms) 

## 2. Replace partial Danish letters in 'neighbourhood_cleansed'

In [4]:
# Define the replacements
replacements = {
    'Nrrebro': 'Nørrebro',
    'sterbro': 'Østerbro',
    'Amager st': 'Amager Øst',
    'Vanlse': 'Vanløse',
    'Brnshj-Husum': 'Brønshøj-Husum'
}

# Replace values in the neighbourhood_cleansed column
listings['neighbourhood_cleansed'] = listings['neighbourhood_cleansed'].replace(replacements)

## 3. Convert date columns to proper date types

In [5]:
# Convert dates
date_columns = [
    'last_scraped',
    'host_since',
    'first_review',
    'last_review',
    'calendar_last_scraped'
]

for col in date_columns:
    # Convert to datetime, handle parsing errors
    listings[col] = pd.to_datetime(listings[col], errors='coerce').dt.date # Keep only the date portion (becomes a Python `date` object which )

## 4. Convert percentage columns to float in range [0, 1]

In [6]:
# Convert percentages to float (remove '%' and divide by 100)
percentage_columns = ['host_response_rate', 'host_acceptance_rate']
for col in percentage_columns:
    listings[col] = (
        listings[col]
        .str.replace('%', '', regex=False)  # Remove percentage symbol
        .astype(float) / 100  # Convert to float and divide by 100
    )

## 5. Convert certain columns to categorical

In [7]:
categorical_columns = [
    'source', 
    'property_type', 
    'room_type', 
    'bathroom_type', 
    'neighbourhood_cleansed'
]
for col in categorical_columns:
    listings[col] = listings[col].astype('category')

## 6. Fix the price column if it's formatted as "$1,865.00"

In [8]:
listings['price'] = listings['price'].str.replace("[$,]", "", regex=True).astype(float) # Remove $ and , and convert to float

## 7. Convert numeric columns with errors to NaN, then fix them

In [9]:
# Ensure all numeric columns are properly cleaned
numeric_columns = ['host_listings_count', 'host_total_listings_count', 'host_response_rate', 'host_acceptance_rate']


In [10]:
# Ensure all numeric columns are properly cleaned
numeric_columns = ['host_listings_count', 'host_total_listings_count', 'host_response_rate', 'host_acceptance_rate']

for col in numeric_columns:
    # Convert to numeric, coercing errors to NaN
    listings[col] = pd.to_numeric(listings[col], errors='coerce')

## 8. Drop empty columns

In [11]:
# Drop empty columns
empty_columns = ['license', 'calendar_updated', 'neighbourhood_group_cleansed']
listings = listings.drop(columns=empty_columns)

## 9. Convert float columns with integer values to int

In [12]:
integer_like_float_columns = [
    'host_listings_count', 
    'host_total_listings_count', 
    'bedrooms', 
    'beds'
]
for col in integer_like_float_columns:
    listings[col] = listings[col].fillna(0).astype(int)

## 10. Clean text columns

In [13]:
text_columns = [
    'name', 
    'description', 
    'neighborhood_overview', 
    'host_name', 
    'host_location', 
    'host_about', 
    'host_response_time', 
    'bathrooms_text',
    'amenities',
]

def remove_br_and_double_quotes(text):
    if not isinstance(text, str):
        return text
    # Replace all literal double quotes with nothing
    return text.replace('<br />', ' ').replace('"', '')

for col in text_columns:
    listings[col] = (
            listings[col]
            .str.replace(r'[\r\n]+', ' ', regex=True) # Replace newlines with spaces
            .str.replace('\u2028', ' ') # Replace unicode line separator with space
            .str.replace('\u2029', ' ') # Replace unicode paragraph separator with space
            .str.replace('\u2019', "'", regex=False) # Replace unicode right single quotation mark with apostrophe
            .str.replace('\u2013', "-", regex=False) # Replace unicode en dash with hyphen
            .str.replace(r'[“”]', '', regex=True) # Remove fancy double quotes
            .str.strip() # Strip leading/trailing whitespace (example: "  hello  " -> "hello")
            .apply(remove_br_and_double_quotes) # Remove <br /> and double quotes (example: "hello<br />" -> hello)
        )

In [14]:
import re

# Extract all unique unicode characters from the amenities column
unicode_chars = set()
for amenities in listings['amenities'].dropna():
    unicode_chars.update(re.findall(r'\\u[0-9a-fA-F]{4}', amenities))

print(unicode_chars)

{'\\u00c3', '\\u00c6', '\\u201c', '\\u20ac', '\\u00d8', '\\u201d', '\\u2019', '\\u00ed', '\\u00c5', '\\u00e6', '\\u00a0', '\\u00e5', '\\u00c4', '\\u00e9', '\\u00e8', '\\u2013', '\\u00f6', '\\u00f8'}


## 11. Replace NaN with None for Postgres-friendly output

In [15]:
listings = listings.where(pd.notnull(listings), None)  # Replace NaN with None, which is converted to NULL in PostgreSQL

## 12. List to plain text

In [16]:
list_columns = ['host_verifications']
for col in list_columns:
    listings[col] = listings[col].apply(lambda x: ', '.join(eval(x)) if pd.notnull(x) else '')

# Export

In [17]:
import csv

In [18]:
listings.to_csv(
    'cleaned_data/cleaned_listings.csv',
    index=False,
    quoting=csv.QUOTE_MINIMAL,  # Use the csv module's quoting options
    encoding='utf-8'
)

In [19]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20909 entries, 0 to 20908
Data columns (total 73 columns):
 #   Column                                        Non-Null Count  Dtype   
---  ------                                        --------------  -----   
 0   id                                            20909 non-null  int64   
 1   listing_url                                   20909 non-null  object  
 2   scrape_id                                     20909 non-null  int64   
 3   last_scraped                                  20909 non-null  object  
 4   source                                        20909 non-null  category
 5   name                                          20909 non-null  object  
 6   description                                   20235 non-null  object  
 7   neighborhood_overview                         8984 non-null   object  
 8   picture_url                                   20909 non-null  object  
 9   host_id                                       2090